# Data Exploration and Profiling Notebook

## Introduction
This notebook is designed to help users perform data exploration and profiling with ease. By following the steps provided, you can load a dataset, generate detailed reports, and visualize your data. This process will help you understand the structure, relationships, and key characteristics of your data, facilitating better decision-making for further analysis or modeling.

Let's get started!


### Install Required Package
In this step, we will install the `ydata-profiling` package, which is essential for generating data profiles.


In [51]:
!pip install --user ydata-profiling
!pip install --user pyod


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.9 MB/s eta 0:00:0000:0100:01
  Created wheel for pyod: filename=pyod-1.1.3-py3-none-any.whl size=190251 sha256=d854932c35aa3ef43211c8ec2ec44f0e1aef2b9b5a7f094b511460168bbed563
  Stored in directory: /tmp/pip/wheels/0e/11/e3/9ceec265e8aed33fede556a3ec9f6fec373776d2c3a6e56f06
Successfully built pyod

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


---
### Import Libraries
Here, we import all the necessary libraries for data analysis and visualization. This includes pandas for data manipulation, matplotlib and seaborn for plotting, and other utility libraries.


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import os
import matplotlib
import sys
import site
import logging

%matplotlib inline
sys.path.append(site.getusersitepackages())

from ydata_profiling import ProfileReport
from scipy.stats import norm
from scipy import stats
from ipywidgets import widgets, Layout
from keboola.component import CommonInterface

warnings.filterwarnings('ignore')

---
### Selecting a Dataset

In this step, we will list the tables that users have loaded into the workspace using the table input mapping. Users can then select the dataset they want to use for data profiling and exploration.

The input datasets are loaded using the Keboola Common Interface, which allows seamless interaction with the data tables defined in the workspace.


In [17]:
# Initialize CommonInterface
ci = CommonInterface()

# Load input tables
input_tables = ci.get_input_tables_definitions()

# List all CSV files in the input tables directory
table_list = []
for table in input_tables:
    table_list.append(table.full_path)

# Create a dropdown widget for selecting a table
if table_list:
    logging.info("Select the dataset you want to use from the dropdown.")
    tables = widgets.Dropdown(options=table_list, value=table_list[0],
                              description='Table:', disabled=False)
    display(tables)
else:
    logging.warning("No tables found. Please ensure you have loaded tables into the workspace using the table input mapping.")


Select the dataset you want to use from the dropdown.
Select the dataset you want to use from the dropdown.


Dropdown(description='Table:', options=('/data/in/tables/configData.csv', '/data/in/tables/TITANIC.csv'), valu…

### Load Selected Dataset
Once you have selected a dataset from the dropdown, this cell reads the CSV file into a pandas DataFrame and generates a profile report using the `ydata-profiling` package.


In [31]:
data = pd.read_csv(tables.value)
profile = ProfileReport(data)
display(data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


---
## Generate Profile Report
This cell generates a comprehensive HTML report of the selected dataset. 

**Warning**: For large datasets, this process may consume a significant amount of memory and might fail. Ensure your dataset is of a manageable size.


In [19]:
profile.to_file(output_file ='/data/profile.html')
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

---
## Remove Redundant Columns

### Identify Columns with Only Unique Values
**NOTE:** Such columns are not useful for binary classification models.

In this section, we will identify columns that contain only unique values. These columns are not helpful for binary classification as they do not provide any discriminative power.


In [16]:
uniqueCols = []
for col in data.columns:
    if len(data[col].unique()) == len(data[col]):
        uniqueCols.append(col)
if len(uniqueCols) > 0:
    print('[WARNING]', 'Columns', uniqueCols, 'contain only unique values, consider editting/grouping the data before using in binary classification model.')
    

[WARNING] Columns ['parameters_size', 'parameters_task_u0', 'parameters_type', 'parameters_expirationAfterHours', 'JSON_parentId'] contain only unique values, consider editting/grouping the data before using in binary classification model.


In [24]:
# Identify columns with only unique values
unique_cols = [col for col in data.columns if data[col].nunique() == len(data)]
if unique_cols:
    logging.info(f'[WARNING] Columns {unique_cols} contain only unique values. Consider editing/grouping the data before using in a binary classification model.')
else:
    logging.info('[INFO] No columns with only unique values found.')


[WARNING] Columns ['PassengerId', 'Name'] contain only unique values. Consider editing/grouping the data before using in a binary classification model.
[WARNING] Columns ['PassengerId', 'Name'] contain only unique values. Consider editing/grouping the data before using in a binary classification model.


### Drop Columns You Don't Want to Use
**NOTE:** You can select multiple columns.

Below is an interactive widget that allows you to select and drop columns that you do not wish to use in your analysis.


In [26]:
# Create a dropdown widget for selecting columns to drop
print("Select columns you want to drop:")
cols_to_drop = widgets.SelectMultiple(
    options=data.columns,
    rows=10,
    description='Columns:',
    disabled=False
)
display(cols_to_drop)

Select columns you want to drop:


SelectMultiple(description='Columns:', options=('PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Si…

### Apply Column Drop
Execute the cell below to drop the selected columns from the dataset.


In [32]:
# Function to drop selected columns
def drop_selected_columns(df, cols):
    cols_to_drop_list = list(cols.get_interact_value())
    if not cols_to_drop_list:
        logging.info('[INFO] All columns will be kept in the data-frame for further exploration.')
    else:
        df.drop(columns=cols_to_drop_list, inplace=True)
        for col in cols_to_drop_list:
            logging.info(f'[INFO] Column {col} dropped.')

# Apply the function with the selected columns
drop_selected_columns(data, cols_to_drop)


[INFO] Column PassengerId dropped.
[INFO] Column PassengerId dropped.


---
## Solve Duplicates
**Drop:** Will drop the duplicate rows.  
**Keep:** Will keep the duplicate rows in the dataset.

In this section, we will identify duplicate rows in the dataset and provide an option to either drop or keep them.


In [34]:
# Count the number of duplicate rows
duplicates_cnt = len(data) - len(data.drop_duplicates())
logging.info(f'[INFO] There are [{duplicates_cnt}] duplicates in the dataset...')

# Create a toggle button for dropping or keeping duplicates
drop_duplicates = 'Keep'
if duplicates_cnt > 0:
    drop_duplicates = widgets.ToggleButtons(
        options=['Drop', 'Keep'],
        description='Duplicates:',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Duplicate rows will be dropped', 'Duplicate rows will be kept'],
        value='Keep'
    )
    display(drop_duplicates)
else:
    logging.info('[INFO] No duplicates found in the dataset.')

[INFO] There are [5] duplicates in the dataset...
[INFO] There are [5] duplicates in the dataset...


ToggleButtons(button_style='info', description='Duplicates:', index=1, options=('Drop', 'Keep'), tooltips=('Du…

### Apply Duplicate Handling
Execute the cell below to apply the chosen option for handling duplicates.


In [35]:
# Apply the chosen option for handling duplicates
if drop_duplicates.value == 'Drop':
    data = data.drop_duplicates()
    logging.info('[INFO] Duplicate rows have been dropped.')
else:
    logging.info('[INFO] Duplicate rows have been kept.')

# Re-check the number of duplicate rows
duplicates_cnt = len(data) - len(data.drop_duplicates())
logging.info(f'[DONE] There are [{duplicates_cnt}] duplicates remaining in the dataset.')


[INFO] Duplicate rows have been dropped.
[INFO] Duplicate rows have been dropped.
[DONE] There are [0] duplicates remaining in the dataset.
[DONE] There are [0] duplicates remaining in the dataset.


---
## Encoding of Categorical Variables

Label encoding has the advantage that it is straightforward but it has the disadvantage that the numeric values can be “misinterpreted” by the algorithms. For example, the value of 0 is obviously less than the value of 4 but does that really correspond to the data set in real life?

In this section, we will:
1. Identify different types of columns in the dataset (date, categorical, numeric).
2. Handle missing values.
3. Apply one-hot encoding to categorical columns.
4. Apply label encoding to selected categorical columns.

Let's start by identifying the column types.


In [36]:
# Function to identify data types
def getDataTypes(data):
    allColumns = list(data.columns)
    datePreds = []
    categoricalPreds = []
    numericPreds = []
    
    for predictor in allColumns:
        if data[predictor].dtype == 'object':
            try:
                pd.to_datetime(data[predictor])
                datePreds.append(predictor)
            except:
                categoricalPreds.append(predictor) 
        elif 'datetime' in str(data[predictor].dtype):
            datePreds.append(predictor)
        else:
            numericPreds.append(predictor)
    
    print('=====================================')
    print('Dataset contains the following data types:')
    print('Date variables:', '[', len(datePreds), ']', datePreds)
    print('-------------------------------------')
    print('Categorical variables:', '[', len(categoricalPreds), ']', categoricalPreds)
    print('-------------------------------------')
    print('Numeric variables:', '[', len(numericPreds), ']', numericPreds)
    print('=====================================')

    return datePreds, categoricalPreds, numericPreds


In [37]:
datePreds, categoricalPreds, numericPreds = getDataTypes(data)

Dataset contains the following data types:
Date variables: [ 0 ] []
-------------------------------------
Categorical variables: [ 5 ] ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
-------------------------------------
Numeric variables: [ 6 ] ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


### One-Hot Encoding

A common approach to encode categorical variables is called one-hot encoding. This method converts each category value into a new column and assigns a 1 or 0 (True/False) value to the column. This prevents misinterpretation of values by the algorithm but adds more columns to the dataset.

Select the categorical columns you want to one-hot encode.


In [38]:
# Widget for selecting columns for one-hot encoding
cols_to_ohe = widgets.SelectMultiple(
    options=categoricalPreds,
    rows=10,
    description='Columns:',
    disabled=False
)
display(cols_to_ohe)

SelectMultiple(description='Columns:', options=('Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'), rows=10, value=…

#### Apply One-Hot Encoding
Execute the cell below to apply one-hot encoding to the selected columns.


In [39]:
# Apply one-hot encoding
data = pd.get_dummies(data, columns=list(cols_to_ohe.value))
logging.info('[INFO] One-hot encoding applied to selected columns.')
datePreds, categoricalPreds, numericPreds = getDataTypes(data)


[INFO] One-hot encoding applied to selected columns.
[INFO] One-hot encoding applied to selected columns.
Dataset contains the following data types:
Date variables: [ 0 ] []
-------------------------------------
Categorical variables: [ 4 ] ['Name', 'Ticket', 'Cabin', 'Embarked']
-------------------------------------
Numeric variables: [ 8 ] ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male']


### Label Encoding

Label encoding converts each value in a column to a number. While straightforward, it can cause the numeric values to be misinterpreted as having an inherent order.

Select the categorical columns you want to label encode.


In [40]:
# Widget for selecting columns for label encoding
cols_to_le = widgets.SelectMultiple(
    options=categoricalPreds,
    rows=10,
    description='Columns:',
    disabled=False
)
display(cols_to_le)


SelectMultiple(description='Columns:', options=('Name', 'Ticket', 'Cabin', 'Embarked'), rows=10, value=())

#### Apply Label Encoding
Execute the cell below to apply label encoding to the selected columns.


In [41]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoding
for col in list(cols_to_le.value):
    lb_make = LabelEncoder()
    data[col + '_code'] = lb_make.fit_transform(data[col])
    data.drop(col, axis=1, inplace=True)

logging.info('[INFO] Label encoding applied to selected columns.')

datePreds, categoricalPreds, numericPreds = getDataTypes(data)
data.head()


[INFO] Label encoding applied to selected columns.
[INFO] Label encoding applied to selected columns.
Dataset contains the following data types:
Date variables: [ 0 ] []
-------------------------------------
Categorical variables: [ 0 ] []
-------------------------------------
Numeric variables: [ 12 ] ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Name_code', 'Ticket_code', 'Cabin_code', 'Embarked_code']


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Name_code,Ticket_code,Cabin_code,Embarked_code
0,0,3,22.0,1,0,7.2500,0,1,108,523,147,2
1,1,1,38.0,1,0,71.2833,1,0,190,596,81,0
2,1,3,26.0,0,0,7.9250,1,0,353,669,147,2
3,1,1,35.0,1,0,53.1000,1,0,272,49,55,2
4,0,3,35.0,0,0,8.0500,0,1,15,472,147,2


---
## Solve Missing Values

### Identify Missing Values

In this section, we will identify the missing values in the dataset. This will help us understand the extent of missing data and decide on an appropriate action to handle it.


In [42]:
# Function to identify missing values
def getMissing(data):
    missing_cnt = data.isna().sum().sum()
    missing_pct = missing_cnt / (len(data.columns) * len(data))     
    missing_out = data.isna().sum()
    
    print('=====================================')
    print(f'Total missing cells: [{missing_cnt}]')
    print(f'Percentage of missing cells: [{missing_pct:.2%}]')
    print('=====================================')
    print('Count of missing cells per column:')
    print(missing_out)
    print('=====================================')
    print('-------------------------------------')

# Identify missing values in the dataset
getMissing(data)


Total missing cells: [177]
Percentage of missing cells: [1.66%]
Count of missing cells per column:
Survived           0
Pclass             0
Age              177
SibSp              0
Parch              0
Fare               0
Sex_female         0
Sex_male           0
Name_code          0
Ticket_code        0
Cabin_code         0
Embarked_code      0
dtype: int64
-------------------------------------


### Decide How to Handle Missing Values

Choose a missing action from the following options:
- **"drop"**: Drop rows with missing values in the selected column(s).
- **"replace"**: Replace missing numeric values with the MEAN and missing categorical values with a new category named "Undefined" for the selected columns.
- **"replaceNumeric"**: Replace missing numeric values with the MEAN value for the selected columns.
- **"replaceCategorical"**: Replace missing categorical values with a new category named "Undefined" for the selected columns.
- **"None"**: Ignore missing values.

<h3><font color="red">↓↓↓ Execute the cell below and choose how to solve missing values ↓↓↓</font></h3>


In [43]:
# Display widgets to choose how to handle missing values
if data.isna().sum().sum() > 0:
    MISSING_ACTION = widgets.ToggleButtons(
        options=['None', 'drop', 'replace', 'replaceNumeric', 'replaceCategorical'],
        description='Action:',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        value='None'
    )
    COLUMNS_ACTION = widgets.SelectMultiple(
        options=['ALL COLUMNS'] + list(data.columns),
        description='Columns:',
        ensure_option=True,
        disabled=False,
        rows=15
    )

    display(MISSING_ACTION)
    display(COLUMNS_ACTION)
else:
    logging.info('[INFO] There are no missing values in your dataset.')

ToggleButtons(button_style='info', description='Action:', options=('None', 'drop', 'replace', 'replaceNumeric'…

SelectMultiple(description='Columns:', options=('ALL COLUMNS', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', …

### Apply Missing Values Action

Execute the cell below to apply the chosen action for handling missing values.

<i><b>NOTE:</b> You can select and execute the missing action multiple times.</i><br>
<i>For example, you can first select 'drop' for a specific column and then 'replaceNumeric' for numeric columns you prefer not to drop.</i>


In [44]:
import ast

# Function to handle missing values based on selected action
def solveMissing(data, MISSING_ACTION):
    messageOut = []
    allColumns = list(data.columns)
    datePreds = []
    categoricalPreds = []
    numericPreds = []
    
    for predictor in allColumns:
        if data[predictor].dtype == 'object':
            try:
                pd.to_datetime(data[predictor])
                datePreds.append(predictor)
            except:
                categoricalPreds.append(predictor) 
        elif 'datetime' in str(data[predictor].dtype):
            datePreds.append(predictor)
        else:
            numericPreds.append(predictor)
    
    if 'None' in MISSING_ACTION[:4]:
        messageOut.append('Not solving any columns.')
        
    if MISSING_ACTION == "replaceAll":
        messageOut.append('Replacing missing values in all columns:')
        for col in allColumns:
            if data[col].isna().sum() > 0:
                if col in numericPreds:
                    data[col].fillna(data[col].mean(), inplace=True)
                else:
                    data[col].fillna('REPLACED-Undefined', inplace=True)
                messageOut.append(col)
            
    elif "replaceNumeric" in MISSING_ACTION:
        messageOut.append('Replacing missing values in NUMERIC columns:')
        for col in numericPreds:
            if data[col].isna().sum() > 0:
                data[col].fillna(data[col].mean(), inplace=True)            
                messageOut.append(col)
            
    elif "replaceCategorical" in MISSING_ACTION:
        messageOut.append('Replacing missing values in CATEGORICAL columns:')
        for col in categoricalPreds:
            if data[col].isna().sum() > 0:
                data[col].fillna('REPLACED-Undefined', inplace=True)
                messageOut.append(col)
    
    elif "replace" in MISSING_ACTION:
        messageOut.append('Replacing missing values in selected columns.')
        colsToReplace = ast.literal_eval(MISSING_ACTION.replace("replace", ""))
        for col in colsToReplace:
            if col in categoricalPreds:
                if data[col].isna().sum() > 0:
                    data[col].fillna('REPLACED-Undefined', inplace=True)
                    messageOut.append(col)
            else:
                if data[col].isna().sum() > 0:
                    data[col].fillna(data[col].mean(), inplace=True)            
                    messageOut.append(col)
                        
    if MISSING_ACTION == 'dropAll':
        messageOut.append('Dropping missing values in all columns.')
        data.dropna(inplace=True)
            
    elif "drop" in MISSING_ACTION[:4]:
        messageOut.append('Dropping missing values in selected columns.')
        colsToDrop = ast.literal_eval(MISSING_ACTION.replace("drop", ""))
        data.dropna(subset=colsToDrop, inplace=True)
        messageOut.append(colsToDrop)
    
    if len(messageOut) == 0:
        messageOut.append('[INFO] There is nothing to do for selected action.')
    print(messageOut)
    return data

# Apply the chosen action for handling missing values
missing_action_value = MISSING_ACTION.value
columns_action_value = list(COLUMNS_ACTION.value)
if 'ALL COLUMNS' in columns_action_value:
    missing_action_concat = missing_action_value + 'All'
else:
    missing_action_concat = missing_action_value + str(columns_action_value)

data = solveMissing(data, missing_action_concat)


['Replacing missing values in NUMERIC columns:', 'Age']


In [45]:
display(data.head())

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Name_code,Ticket_code,Cabin_code,Embarked_code
0,0,3,22.0,1,0,7.2500,0,1,108,523,147,2
1,1,1,38.0,1,0,71.2833,1,0,190,596,81,0
2,1,3,26.0,0,0,7.9250,1,0,353,669,147,2
3,1,1,35.0,1,0,53.1000,1,0,272,49,55,2
4,0,3,35.0,0,0,8.0500,0,1,15,472,147,2


---
## Check the basic details of dataset

In [48]:
dataDescribe = data.describe(include='all').transpose()
dataDescribe['ColumnName'] = dataDescribe.index

display(dataDescribe)

,count,mean,std,min,25%,50%,75%,max,ColumnName
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.000000,1.0,1.0000,Survived
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.000000,3.0,3.0000,Pclass
Age,891.0,29.699118,13.002015,0.42,22.0000,29.699118,35.0,80.0000,Age
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.000000,1.0,8.0000,SibSp
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.000000,0.0,6.0000,Parch
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.454200,31.0,512.3292,Fare
Sex_female,891.0,0.352413,0.477990,0.00,0.0000,0.000000,1.0,1.0000,Sex_female
Sex_male,891.0,0.647587,0.477990,0.00,0.0000,1.000000,1.0,1.0000,Sex_male
Name_code,891.0,445.000000,257.353842,0.00,222.5000,445.000000,667.5,890.0000,Name_code
Ticket_code,891.0,338.528620,200.850657,0.00,158.5000,337.000000,519.5,680.0000,Ticket_code


---
## Anomaly Detection

### Skewness and Kurtosis Analysis (of Numeric Variables)

**Skewness** is a measure of symmetry, or more precisely, the lack of symmetry. A distribution, or data set, is symmetric if it looks the same to the left and right of the center point.

**Kurtosis** is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution. Data sets with high kurtosis tend to have heavy tails, or outliers. Data sets with low kurtosis tend to have light tails, or lack of outliers.

In this section, we will:
1. Calculate skewness and kurtosis for numeric variables.
2. Identify columns that are highly skewed or peaked.

Execute the cell below to perform the skewness and kurtosis analysis.


In [49]:
# Function to calculate skewness and kurtosis
def getSkewKurt(data):
    dataSkewKurt = pd.DataFrame({"ColumnName": [], "Skewness": [], "Kurtosis": []})
    for col in data.columns:
        colSkew = data[col].skew()
        colKurt = data[col].kurt()
        dataSkewKurtTemp = pd.DataFrame({"ColumnName": [col], "Skewness": [colSkew], "Kurtosis": [colKurt]})
        dataSkewKurt = dataSkewKurt.append(dataSkewKurtTemp, ignore_index=True)
        if colSkew < -2 or colSkew > 2:
            print(f'[WARNING] The {col} column is highly skewed. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.\n')
        if colKurt < -3 or colKurt > 3:
            print(f'[WARNING] The {col} column is highly peaked. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.\n')
    return dataSkewKurt

# Calculate skewness and kurtosis for numeric variables
dataSkewKurt = getSkewKurt(data[numericPreds])

# Button to view skewness and kurtosis values
button = widgets.Button(description="View Skewness and Kurtosis values for all columns",
                        layout=Layout(width='50%'))
output = widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        display(dataSkewKurt)

button.on_click(on_button_clicked)

[WARNING] The SibSp column is highly skewed. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The SibSp column is highly peaked. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The Parch column is highly skewed. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The Parch column is highly peaked. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The Fare column is highly skewed. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The Fare column is highly peaked. This can affect the performance of binary classification. Check the dataset if the share of outliers is acceptable.

[WARNING] The Cabin_code colum

Button(description='View Skewness and Kurtosis values for all columns', layout=Layout(width='50%'), style=Butt…

Output()

### Identify Multivariate Outliers for Numeric Columns

Outliers in the dataset can significantly affect the performance of any binary classification algorithm.

**There is no rule of thumb for what share of outliers is acceptable. What is crucial is to be aware of the outliers present in the dataset.**

**Method 1 - Standard Deviation (STD)**:
For every column, calculate the mean and standard deviation (StD). Each data point is then compared with the mean as follows:
`IF ABS(DATA_POINT - MEAN) > 2*StD THEN OUTLIER ELSE NOT_OUTLIER`

**Method 2 - K-Nearest Neighbor (KNN)**:
The KNN method uses the distance to its k-th nearest neighbor as the outlying score. PyOD supports three KNN detectors: largest, mean, and median, which use the distance of the k-th neighbor, the average of all the k neighbors, and the median distance to k neighbors as the outlying score, respectively.

Execute the cell below to identify outliers using both methods.


In [52]:
from pyod.models.knn import KNN

# Function to calculate outliers based on standard deviation
def getNumericOutliers(data):
    print('[INFO] Calculating outliers based on standard deviation')
    dataOutlier = data.copy()
    for col in dataOutlier:
        dataOutlier[col + '_mean'] = dataOutlier[col].mean()
        dataOutlier[col + '_std'] = dataOutlier[col].std()
        dataOutlier[col + '_outlier'] = dataOutlier[[col, col + '_mean', col + '_std']].apply(
            lambda x: 1 if abs(x[col] - x[col + '_mean']) > (x[col + '_std'] * 2) else 0, axis=1)

    outlierCols = [col for col in dataOutlier.columns if '_outlier' in col]
    dataOutlier['StdOutlier'] = dataOutlier[outlierCols].apply(lambda x: 1 if x.sum() > 1 else 0, axis=1)
    dataOutlier.drop(columns=[col for col in dataOutlier.columns if '_mean' in col or '_std' in col or '_outlier' in col], inplace=True)
    
    outlier_fraction = dataOutlier['StdOutlier'].mean()
    print(f'[INFO] Done. Count of outliers based on Std: {dataOutlier["StdOutlier"].sum()}')
    print(f'Share of outlier rows: {outlier_fraction:.2%}')
    
    return dataOutlier, outlier_fraction

# Function to calculate outliers based on KNN
def getKnnOutliers(X, fraction):
    print('[INFO] Calculating outliers with KNN model')
    clf = KNN(contamination=fraction)
    clf.fit(X)
    
    # Predict raw anomaly score
    scores_pred = clf.decision_function(X) * -1

    # Prediction of a datapoint category outlier or inlier
    y_pred = clf.predict(X).tolist()
    
    print('[INFO] Done. Count of outliers based on KNN:', sum(y_pred))
    print('Share of outlier rows:', f'{sum(y_pred) / len(y_pred):.2%}')
    
    return y_pred

# Calculate outliers based on standard deviation
dataNumericOutlier, fraction = getNumericOutliers(data[numericPreds])

# Prepare data for KNN outlier detection
dataKnn = data[numericPreds].astype('float64')
X = dataKnn.values

# Calculate outliers based on KNN
dataKnnOutlier = getKnnOutliers(X, fraction)
dataKnn['KnnOutlier'] = dataKnnOutlier

# Combine outlier results with the main dataset
data['StdOutlier'] = dataNumericOutlier['StdOutlier']
data['KnnOutlier'] = dataKnn['KnnOutlier']
print('Columns [StdOutlier, KnnOutlier] were added to the dataset.')


[INFO] Calculating outliers based on standard deviation
[INFO] Done. Count of outliers based on Std: 76
Share of outlier rows: 8.53%
[INFO] Calculating outliers with KNN model
[INFO] Done. Count of outliers based on KNN: 67
Share of outlier rows: 7.52%
Columns [StdOutlier, KnnOutlier] were added to the dataset.


### Explore Outliers

Explore the columns with standard deviation outliers.

Execute the cell below to view the rows identified as outliers by standard deviation.


In [53]:
# Display rows with standard deviation outliers
dataNumericOutlier[dataNumericOutlier['StdOutlier'] == 1]


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Name_code,Ticket_code,Cabin_code,Embarked_code,StdOutlier
7,0,3,2.000000,3,1,21.0750,0,1,624,395,147,2,1
11,1,1,58.000000,0,0,26.5500,1,0,95,38,49,2,1
16,0,3,2.000000,4,1,29.1250,0,1,682,480,147,1,1
27,0,1,19.000000,3,2,263.0000,0,1,260,95,63,2,1
31,1,1,29.699118,1,0,146.5208,1,0,776,586,41,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,1,2,1.000000,0,2,37.0042,0,1,503,618,147,0,1
829,1,1,62.000000,0,0,80.0000,1,0,785,32,21,3,1
846,0,3,29.699118,8,2,69.5500,0,1,720,568,147,2,1
850,0,3,4.000000,4,2,31.2750,0,1,22,333,147,2,1


### Explore Description of KNN Outliers

Execute the cell below to view the descriptive statistics of the rows identified as KNN outliers.


In [54]:
# Display description of KNN outliers
dataKnn[dataKnn['KnnOutlier'] == 1].describe().transpose()


,count,mean,std,min,25%,50%,75%,max
Survived,67.0,0.656716,0.478389,0.00,0.0000,1.0,1.00000,1.0000
Pclass,67.0,1.000000,0.000000,1.00,1.0000,1.0,1.00000,1.0000
Age,67.0,34.037507,15.939361,0.92,23.5000,31.0,42.50000,80.0000
SibSp,67.0,0.522388,0.785440,0.00,0.0000,0.0,1.00000,3.0000
Parch,67.0,0.656716,0.913738,0.00,0.0000,0.0,1.00000,4.0000
Fare,67.0,150.954415,107.660277,0.00,77.7771,134.5,211.41875,512.3292
Sex_female,67.0,0.582090,0.496938,0.00,0.0000,1.0,1.00000,1.0000
Sex_male,67.0,0.417910,0.496938,0.00,0.0000,0.0,1.00000,1.0000
Name_code,67.0,419.611940,290.486737,14.00,175.5000,305.0,695.00000,877.0000
Ticket_code,67.0,402.955224,238.814904,0.00,118.0000,502.0,604.50000,680.0000


### Drop/Replace/Keep Outliers

If you have identified outliers in your dataset, you can choose to drop, replace, or keep them.

<h3><font color="red">↓↓↓ Execute the cell below and choose how to handle outliers ↓↓↓</font></h3>


In [55]:
# Display widgets to choose how to handle outliers
if len(data[data['StdOutlier'] == 1]) > 0 or len(data[data['KnnOutlier'] == 1]) > 0:
    OUTLIER_ACTION = widgets.ToggleButtons(
        options=['Keep', 'Drop'],
        description='Action:',
        disabled=False,
        button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
        value='Keep'
    )

    DETAIL_ACTION = widgets.ToggleButtons(
        options=['All', 'Knn', 'Std'],
        description='For:',
        disabled=False,
        button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
        value='All'
    )
    display(OUTLIER_ACTION)
    display(DETAIL_ACTION)
else:
    print('[INFO] There are no outliers in your dataset.')


ToggleButtons(button_style='info', description='Action:', options=('Keep', 'Drop'), value='Keep')

ToggleButtons(button_style='success', description='For:', options=('All', 'Knn', 'Std'), value='All')

### Apply Outlier Handling Action

Execute the cell below to apply the chosen action for handling outliers.


In [56]:
# Function to handle outliers based on selected action
def solveOutliers(data, outlier_action_value, outlier_action_type):
    messageOut = []
    
    if outlier_action_value == 'Keep':
        messageOut.append('All outliers were kept in the dataset.')

    elif outlier_action_value == 'Drop':
        if outlier_action_type == 'All':
            data = data[(data['StdOutlier'] == 0) & (data['KnnOutlier'] == 0)]
            messageOut.append('All outliers were dropped.')
            
        elif outlier_action_type == 'Knn':
            data = data[data['KnnOutlier'] == 0]
            messageOut.append('KNN outliers were dropped.')
            
        elif outlier_action_type == 'Std':
            data = data[data['StdOutlier'] == 0]
            messageOut.append('STD outliers were dropped.')   
    
    print(messageOut)
    return data

# Apply the chosen action for handling outliers
if len(data[data['StdOutlier'] == 1]) > 0 or len(data[data['KnnOutlier'] == 1]) > 0:
    outlier_action_value = OUTLIER_ACTION.value
    outlier_action_type = DETAIL_ACTION.value
    data = solveOutliers(data, outlier_action_value, outlier_action_type)


['KNN outliers were dropped.']


In [58]:
display(data.head())

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Name_code,Ticket_code,Cabin_code,Embarked_code,StdOutlier,KnnOutlier
0,0,3,22.0,1,0,7.2500,0,1,108,523,147,2,0,0
1,1,1,38.0,1,0,71.2833,1,0,190,596,81,0,0,0
2,1,3,26.0,0,0,7.9250,1,0,353,669,147,2,0,0
3,1,1,35.0,1,0,53.1000,1,0,272,49,55,2,0,0
4,0,3,35.0,0,0,8.0500,0,1,15,472,147,2,0,0


---
## Export transformed data-set

In [59]:
try:
    data.drop('StdOutlier', axis=1, inplace=True)
except:
    pass
try:
    data.drop('KnnOutlier', axis=1, inplace=True)
except:
    pass

data.to_csv('/data/data_transformed.csv', index=False)
logging.info('Transformed dataset was loaded to the root folder of this workspace...')

Transformed dataset was loaded to the root folder of this workspace...
Transformed dataset was loaded to the root folder of this workspace...
